In [1]:


# import modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dill
from scipy.stats import pearsonr
import math
from scipy.stats import chi2_contingency


from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
##### Import of tables into dataframes
dfLieux = pd.read_csv('20220906_table_lieux.csv', sep=',')

dfUsagers = pd.read_csv('20220906_table_usagers.csv', sep=',')
dfUsagers = dfUsagers.drop(["Unnamed: 0.1","Unnamed: 0"], axis = 1)

dfVehicules = pd.read_csv('20220906_table_vehicules.csv', sep=',')
dfVehicules = dfVehicules.drop(["Unnamed: 0.1","Unnamed: 0"], axis = 1)

dfCarac = pd.read_csv('20220906_table_caracteristiques.csv', sep=',')
dfCarac = dfCarac.drop(["Unnamed: 0.1","Unnamed: 0"], axis = 1)

dfPool = pd.read_csv('20221023_table_pool.csv')
dfPool = dfPool.drop(["Unnamed: 0"], axis = 1)

C:\Users\possa\AppData\Local\Temp\ipykernel_12364\3967809590.py:2: DtypeWarning: Columns (4,9,10,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dfLieux = pd.read_csv('20220906_table_lieux.csv', sep=',')
C:\Users\possa\AppData\Local\Temp\ipykernel_12364\3967809590.py:4: DtypeWarning: Columns (10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfUsagers = pd.read_csv('20220906_table_usagers.csv', sep=',')
C:\Users\possa\AppData\Local\Temp\ipykernel_12364\3967809590.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfVehicules = pd.read_csv('20220906_table_vehicules.csv', sep=',')
C:\Users\possa\AppData\Local\Temp\ipykernel_12364\3967809590.py:10: DtypeWarning: Columns (6,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCarac = pd.read_csv('20220906_table_caracteristiques.csv', sep=',')
C:\Users\possa\AppData\Local\Temp

In [3]:
# ##### Computing new variables
# Computing date variable
dfCarac['date'] = dfCarac['jour'].astype(str) + '-' + dfCarac['mois'].astype(str) + '-' + dfCarac['an'].astype(str)
dfCarac['date'] = pd.to_datetime(dfCarac['date']);

C:\Users\possa\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30-11-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\possa\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28-11-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\possa\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29-11-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\possa\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27-11-2019' in DD/MM/YYYY format. Provide format or specif

In [4]:
# Computing months with categorical labels
dfCarac['mois_label'] = dfCarac['mois']
dfCarac['mois_label'] = dfCarac['mois_label'].replace(to_replace=np.arange(1, 13, 1), 
                                                      value=['jan', 'fev', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])

# Days of the week
dfCarac['weekday'] = dfCarac['date'].dt.weekday

# Hour of the day
dfCarac['hrmn'] = dfCarac['hrmn'].replace('\:', '', regex=True).astype(int)
dfCarac['hour'] = dfCarac['hrmn']//100

# Year of accident
dfCarac['year'] = dfCarac['date'].dt.year

# Adding the year variable to dfUsagers dataframe
dfUsagers = dfUsagers.merge(right=dfCarac[['Num_Acc', 'year']], on='Num_Acc')

In [5]:
# Age of people during the accident (removing ages above 99, could be completion issues and there are very few values)
dfUsagers['age'] = dfUsagers.year - dfUsagers.an_nais
dfUsagers.loc[dfUsagers['age'] > 99, 'age'] = np.nan

In [46]:
# Most injured person 
dfgravMaxAcc =pd.DataFrame(dfUsagers[['Num_Acc', 'grav2']].groupby(['Num_Acc']).max())#.rename({'grav2':'gravMaxAcc'}, axis=1)
#dfUsagers = dfUsagers.merge(right=dfgravMaxAccUsagers, how='left', on='Num_Acc')

dfgravMaxAccUsagers=dfUsagers.merge(right=dfgravMaxAcc, how='inner', on=['Num_Acc','grav2'])
dfgravMaxAccUsagers=dfgravMaxAccUsagers.rename({'grav2':'gravMaxAcc'}, axis=1)

In [7]:
dfgravMaxAccUsagers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1282398 entries, 0 to 1282397
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Num_Acc      1282398 non-null  int64  
 1   place        1163672 non-null  float64
 2   catu         1282398 non-null  int64  
 3   grav         1282398 non-null  int64  
 4   sexe         1282398 non-null  int64  
 5   trajet       1282136 non-null  float64
 6   secu         1100197 non-null  float64
 7   locp         1254120 non-null  float64
 8   actp         1254105 non-null  object 
 9   etatp        1254116 non-null  float64
 10  an_nais      1281074 non-null  float64
 11  num_veh      1282398 non-null  object 
 12  id_vehicule  128188 non-null   object 
 13  secu1        128188 non-null   float64
 14  secu2        128188 non-null   float64
 15  secu3        128188 non-null   float64
 16  gravMaxAcc   1282398 non-null  int64  
 17  year         1282398 non-null  int64  
 18  ag

In [47]:
#la + grande gravité regroupée par accident
# dfM1: si 1 ligne/ acc -->Ok
#       sinon:
# dfM2        regrouper par la personne la + agée, 
# dfM3        ou par la personne la + jeune

dfA = dfgravMaxAccUsagers.groupby('Num_Acc')
dfM1= dfA.filter(lambda x: len(x)==1)  #  HAVING COUNT(*) = 1
dfM1['gravMaxAcc_Oldest_age']     =dfM1['age']
dfM1['gravMaxAcc_Oldest_catu']    =dfM1['catu']
dfM1['gravMaxAcc_Oldest_sexe']    =dfM1['sexe']
dfM1['gravMaxAcc_Oldest_trajet']  =dfM1['trajet']
dfM1['gravMaxAcc_Oldest_locp']    =dfM1['locp']
dfM1['gravMaxAcc_Oldest_actp']    =dfM1['actp']
dfM1['gravMaxAcc_Oldest_locp']    =dfM1['locp']

dfM1['gravMaxAcc_Youngest_age']   =dfM1['age']
dfM1['gravMaxAcc_Youngest_catu']  =dfM1['catu']
dfM1['gravMaxAcc_Youngest_sexe']  =dfM1['sexe']
dfM1['gravMaxAcc_Youngest_trajet']=dfM1['trajet']
dfM1['gravMaxAcc_Youngest_locp']  =dfM1['locp']
dfM1['gravMaxAcc_Youngest_actp']  =dfM1['actp']
dfM1['gravMaxAcc_Youngest_locp']  =dfM1['locp']

#dfA[['Num_Acc', 'place', 'num_veh','grav','catu', 'gravMaxAcc', 'sexe', 'age']].filter(lambda x: len(x) > 1)  #HAVING COUNT(*) > 1

In [48]:
dfM2=dfA.filter(lambda x: len(x)>1)  #  HAVING COUNT(*)>1 -- #378 424 

In [49]:
# dfM2
idxMax=dfM2.groupby(['Num_Acc','gravMaxAcc'])['age'].transform(max)==dfM2['age']  #378424: max / #
dfM2_max1 =dfM2[idxMax]

idxMin=dfM2.groupby(['Num_Acc','gravMaxAcc'])['age'].transform(min)==dfM2['age']  #378424: max / #
dfM2_min1 =dfM2[idxMin]


In [50]:

#occurence unique
dfM2_max1=dfM2_max1.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)==1)
dfM2_max1['gravMaxAcc_Oldest_age']     =dfM2_max1['age']
dfM2_max1['gravMaxAcc_Oldest_catu']    =dfM2_max1['catu']
dfM2_max1['gravMaxAcc_Oldest_sexe']    =dfM2_max1['sexe']
dfM2_max1['gravMaxAcc_Oldest_trajet']  =dfM2_max1['trajet']
dfM2_max1['gravMaxAcc_Oldest_locp']    =dfM2_max1['locp']
dfM2_max1['gravMaxAcc_Oldest_actp']    =dfM2_max1['actp']
dfM2_max1['gravMaxAcc_Oldest_locp']    =dfM2_max1['locp']

#occurence unique
dfM2_min1=dfM2_min1.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)==1)
dfM2_min1['gravMaxAcc_Youngest_age']     =dfM2_min1['age']
dfM2_min1['gravMaxAcc_Youngest_catu']    =dfM2_min1['catu']
dfM2_min1['gravMaxAcc_Youngest_sexe']    =dfM2_min1['sexe']
dfM2_min1['gravMaxAcc_Youngest_trajet']  =dfM2_min1['trajet']
dfM2_min1['gravMaxAcc_Youngest_locp']    =dfM2_min1['locp']
dfM2_min1['gravMaxAcc_Youngest_actp']    =dfM2_min1['actp']
dfM2_min1['gravMaxAcc_Youngest_locp']    =dfM2_min1['locp']



In [41]:
#dfM2_max1.loc[dfM2_max1['Num_Acc']==202000047703]
#dfM2_min1.loc[dfM2_min1['Num_Acc']==202000047703]


,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,secu3,gravMaxAcc,year,age,gravMaxAcc_Youngest_age,gravMaxAcc_Youngest_catu,gravMaxAcc_Youngest_sexe,gravMaxAcc_Youngest_trajet,gravMaxAcc_Youngest_locp,gravMaxAcc_Youngest_actp
1282348,202000047703,1.0,1,4,1,4.0,NaN,-1.0,-1,-1.0,...,-1.0,2,2020,22.0,22.0,1,1,4.0,-1.0,-1


In [51]:
#Qd il existe plusieurs occurences: on prend conducteur(catu=1)
dfM2_max11=dfM2_max1.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)>1)

#dfM2_max11.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)==1)
dfM2_max1_catu1=dfM2_max11.loc[dfM2_max11['catu']==1]

dfM2_max1_catu1['gravMaxAcc_Oldest_age']     =dfM2_max1_catu1['age']
dfM2_max1_catu1['gravMaxAcc_Oldest_catu']    =dfM2_max1_catu1['catu']
dfM2_max1_catu1['gravMaxAcc_Oldest_sexe']    =dfM2_max1_catu1['sexe']
dfM2_max1_catu1['gravMaxAcc_Oldest_trajet']  =dfM2_max1_catu1['trajet']
dfM2_max1_catu1['gravMaxAcc_Oldest_locp']    =dfM2_max1_catu1['locp']
dfM2_max1_catu1['gravMaxAcc_Oldest_actp']    =dfM2_max1_catu1['actp']
dfM2_max1_catu1['gravMaxAcc_Oldest_locp']    =dfM2_max1_catu1['locp']


In [52]:
#Qd il existe plusieurs occurences: on prend conducteur(catu=1)
dfM2_min11=dfM2_min1.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)>1)

#dfM2_min11.groupby(['Num_Acc','gravMaxAcc']).filter(lambda x: len(x)==1)
dfM2_min1_catu1=dfM2_min11.loc[dfM2_min11['catu']==1]

dfM2_min1_catu1['gravMaxAcc_Oldest_age']     =dfM2_min1_catu1['age']
dfM2_min1_catu1['gravMaxAcc_Oldest_catu']    =dfM2_min1_catu1['catu']
dfM2_min1_catu1['gravMaxAcc_Oldest_sexe']    =dfM2_min1_catu1['sexe']
dfM2_min1_catu1['gravMaxAcc_Oldest_trajet']  =dfM2_min1_catu1['trajet']
dfM2_min1_catu1['gravMaxAcc_Oldest_locp']    =dfM2_min1_catu1['locp']
dfM2_min1_catu1['gravMaxAcc_Oldest_actp']    =dfM2_min1_catu1['actp']
dfM2_min1_catu1['gravMaxAcc_Oldest_locp']    =dfM2_min1_catu1['locp']


In [53]:
frames = [dfM1,dfM2_max1,dfM2_min1,dfM2_max1_catu1,dfM2_min1_catu1]
dfAllgravMaxAccUsagers=pd.concat(frames)

In [54]:
dfAllgravMaxAccUsagers.shape

(1198170, 31)

In [32]:
dfAllgravMaxAccUsagers.to_csv(r'C:\Users\possa\Documents\Projets DataScienTEST\Accidents routiers en France\dfAllgravMaxAccUsagers.csv')

In [33]:
#dfgravMaxAcc
dfAllgravMaxAccUsagers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051648 entries, 0 to 1282395
Data columns (total 31 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Num_Acc                     1051648 non-null  int64  
 1   place                       939765 non-null   float64
 2   catu                        1051648 non-null  int64  
 3   grav                        1051648 non-null  int64  
 4   sexe                        1051648 non-null  int64  
 5   trajet                      1051491 non-null  float64
 6   secu                        895702 non-null   float64
 7   locp                        1029504 non-null  float64
 8   actp                        1029494 non-null  object 
 9   etatp                       1029496 non-null  float64
 10  an_nais                     1050792 non-null  float64
 11  num_veh                     1051648 non-null  object 
 12  id_vehicule                 105188 non-null   object 
 1

In [55]:
dfAllgravMaxAccUsagers[['Num_Acc','place','num_veh','gravMaxAcc','age','gravMaxAcc_Oldest_age','gravMaxAcc_Youngest_age']]

,Num_Acc,place,num_veh,gravMaxAcc,age,gravMaxAcc_Oldest_age,gravMaxAcc_Youngest_age
0,201600000001,1.0,A01,3,15.0,15.0,15.0
4,201600000003,NaN,A01,3,46.0,46.0,46.0
5,201600000004,1.0,B02,3,17.0,17.0,17.0
8,201600000006,NaN,A01,3,38.0,38.0,38.0
9,201600000007,2.0,A01,2,15.0,15.0,15.0
...,...,...,...,...,...,...,...
1282348,202000047703,1.0,C01,2,22.0,NaN,22.0
1282365,202000047719,1.0,B01,2,26.0,NaN,26.0
1282382,202000047734,1.0,B01,2,21.0,NaN,21.0
1282388,202000047740,2.0,B01,2,11.0,NaN,11.0


In [56]:
dfUsagers.loc[dfUsagers['Num_Acc']==202000047703]

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,id_vehicule,secu1,secu2,secu3,grav2,year,age
2380373,202000047703,1.0,1,4,1,5.0,NaN,-1.0,-1,-1.0,1989.0,A01,154Â 658Â 424,1.0,0.0,-1.0,2,2020,31.0
2380374,202000047703,1.0,1,1,1,4.0,NaN,-1.0,-1,-1.0,1994.0,B01,154Â 658Â 425,1.0,0.0,-1.0,1,2020,26.0
2380375,202000047703,1.0,1,4,1,4.0,NaN,-1.0,-1,-1.0,1998.0,C01,154Â 658Â 426,1.0,0.0,-1.0,2,2020,22.0


In [57]:
dfAllgravMaxAccUsagers.loc[dfAllgravMaxAccUsagers['Num_Acc']==202000047703][['Num_Acc','place','num_veh','gravMaxAcc','age','gravMaxAcc_Oldest_age','gravMaxAcc_Youngest_age']]

,Num_Acc,place,num_veh,gravMaxAcc,age,gravMaxAcc_Oldest_age,gravMaxAcc_Youngest_age
1282347,202000047703,1.0,A01,2,31.0,31.0,NaN
1282348,202000047703,1.0,C01,2,22.0,NaN,22.0


In [74]:
### Computing all variables as 'is there at least one of'
dfgravMaxAcc_OneByAcc = pd.DataFrame({
             # event exists yes/no by accident
            'Num_Acc'                    : dfAllgravMaxAccUsagers.groupby('Num_Acc')['Num_Acc'].sum().index,
            'gravMaxAcc_Oldest_age'      : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_age'].max(),
            'gravMaxAcc_Oldest_catu'     : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_catu'].max(),
            'gravMaxAcc_Oldest_sexe'     : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_sexe'].max(),
            'gravMaxAcc_Oldest_trajet'   : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_trajet'].max(),
            'gravMaxAcc_Oldest_locp'     : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_locp'].max(),
           # 'gravMaxAcc_Oldest_actp'     : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Oldest_actp'].max(),
            'gravMaxAcc_Youngest_age'    : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_age'].min(),
            'gravMaxAcc_Youngest_catu'   : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_catu'].min(),
            'gravMaxAcc_Youngest_sexe'   : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_sexe'].min(),
            'gravMaxAcc_Youngest_trajet' : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_trajet'].min(),
            'gravMaxAcc_Youngest_locp'   : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_locp'].min(),
            #'gravMaxAcc_Youngest_actp'   : dfAllgravMaxAccUsagers.groupby('Num_Acc')['gravMaxAcc_Youngest_actp'].min()
})

In [75]:
dfgravMaxAcc_OneByAcc.to_csv(r'C:\Users\possa\Documents\Projets DataScienTEST\Accidents routiers en France\dfgravMaxAcc_OneByAcc.csv')

In [76]:
dfgravMaxAcc_OneByAcc.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1054168 entries, 200500000001 to 202000047744
Data columns (total 11 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Num_Acc                     1054168 non-null  int64  
 1   gravMaxAcc_Oldest_age       1050641 non-null  float64
 2   gravMaxAcc_Oldest_catu      1051648 non-null  float64
 3   gravMaxAcc_Oldest_sexe      1051648 non-null  float64
 4   gravMaxAcc_Oldest_trajet    1051491 non-null  float64
 5   gravMaxAcc_Oldest_locp      1029504 non-null  float64
 6   gravMaxAcc_Youngest_age     1049489 non-null  float64
 7   gravMaxAcc_Youngest_catu    1050496 non-null  float64
 8   gravMaxAcc_Youngest_sexe    1050496 non-null  float64
 9   gravMaxAcc_Youngest_trajet  1050322 non-null  float64
 10  gravMaxAcc_Youngest_locp    1028385 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 96.5 MB
